In [1]:
import pandas as pd
import torch
import random
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, Trainer, TrainingArguments
from torch import nn

import numpy as np
import random
import io

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, T5EncoderModel, AdamW, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm import tqdm
import os
from dataclasses import dataclass
from typing import Optional
import ast
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import pandas as pd
import os
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, infer_auto_device_map

from transformers import AutoConfig

import pandas as pd
from sklearn.model_selection import train_test_split
import random
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported



/home/stud/haroonm0/localdisk/conda_envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2024.12.8: Fast Mistral patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.553 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
fact_checks = pd.read_csv('/home/stud/haroonm0/localdisk/Fact_check/dataset/fact_checks.csv')
posts = pd.read_csv('/home/stud/haroonm0/localdisk/Fact_check/dataset/posts.csv')
pairs = pd.read_csv('/home/stud/haroonm0/localdisk/Fact_check/dataset/pairs.csv')


def create_pos_neg_pairs(train_df, merged_data):
    pos_pairs = []
    for _, row in train_df.iterrows():
        pos_pairs.append({
            'post_id': row['post_id'],
            'text_original': row['text_original'],
            'text_translated': row['text_translated'],
            'fact_check_id': row['fact_check_id'],
            'original_claim': row['original_claim'],
            'translated_claim': row['translated_claim'],
            'label': 1  # Positive label
        })

    unique_fact_checks = merged_data.drop_duplicates(subset='fact_check_id')
    fact_check_ids = merged_data['fact_check_id'].unique()

    neg_pairs = []
    for _, row in train_df.iterrows():
        correct_fact_check_id = row['fact_check_id']
        available_fact_check_ids = list(set(fact_check_ids) - {correct_fact_check_id})
        negative_fact_checks = random.sample(available_fact_check_ids, k=min(3, len(available_fact_check_ids)))

        for neg_fact_id in negative_fact_checks:
            neg_fact_check_row = merged_data[merged_data['fact_check_id'] == neg_fact_id].iloc[0]
            neg_pairs.append({
                'post_id': row['post_id'],
                'text_original': row['text_original'],
                'text_translated': row['text_translated'],
                'fact_check_id': neg_fact_id,
                'original_claim': neg_fact_check_row['original_claim'],
                'translated_claim': neg_fact_check_row['translated_claim'],
                'label': 0  # Negative label
            })

    all_pairs = pos_pairs + neg_pairs
    return pd.DataFrame(all_pairs)

def split_data(posts, merged_data, test_size=0.2, val_size=0.5, random_state=42):
    unique_post_ids = posts['post_id'].unique()
    train_ids, temp_ids = train_test_split(unique_post_ids, test_size=test_size, random_state=random_state)
    val_ids, test_ids = train_test_split(temp_ids, test_size=val_size, random_state=random_state)

    train_df = merged_data[merged_data['post_id'].isin(train_ids)]
    val_df = merged_data[merged_data['post_id'].isin(val_ids)]
    test_df = merged_data[merged_data['post_id'].isin(test_ids)]

    return train_df, val_df, test_df


def merge_data(posts, fact_checks, pairs):
    """
    Merge posts and fact_checks using pairs as the bridge.
    """

    posts = posts.drop_duplicates(subset='post_id')
    fact_checks = fact_checks.drop_duplicates(subset='fact_check_id')
    pairs = pairs.drop_duplicates(subset=['post_id', 'fact_check_id'])


    merged_data = pairs.merge(posts, on='post_id', how='left').merge(fact_checks, on='fact_check_id', how='left')


    merged_data.drop(
        columns=['instances_x', 'verdicts', 'ocr_confidence', 'instances_y', 'confidence'],
        inplace=True,
        errors='ignore'
    )

    return merged_data





def pre_process(posts, fact_checks, pairs):


    #fact_checks = pd.read_csv("/home/stud/haroonm0/localdisk/Fact_check/dataset/fact_checks.csv")
    #posts = pd.read_csv("/home/stud/haroonm0/localdisk/Fact_check/dataset/posts.csv")
    #pairs = pd.read_csv("/home/stud/haroonm0/localdisk/Fact_check/dataset/pairs.csv")
    preprocessor = TextPreprocessor()

    if 'text' in posts.columns and 'ocr' in posts.columns:
        posts = replace_text_with_ocr(posts, text_column='text', ocr_column='ocr')
    if 'ocr' in posts.columns:
        posts[['ocr_original', 'ocr_translated', 'ocr_language', 'ocr_confidence']] = posts.apply(
            lambda row: split_text_column(row, 'ocr'), axis=1
        )
    if 'text' in posts.columns:
        posts[['text_original', 'text_translated', 'text_language', 'text_confidence']] = posts.apply(
            lambda row: split_text_column(row, 'text'), axis=1
        )
    fact_checks[['original_claim', 'translated_claim', 'language', 'confidence']] = fact_checks.apply(
        lambda row: split_text_column(row, 'claim'), axis=1
    )

    columns_to_preprocess = [
        ('translated_claim', fact_checks),
        ('text_translated', posts),
        ('ocr_translated', posts),
        ('original_claim', fact_checks),
        ('text_original', posts),
        ('ocr_original', posts)
    ]
    for col, df in columns_to_preprocess:
        if col in df.columns:
            df[col] = df[col].apply(preprocessor.preprocess)

    mergedata = merge_data(posts, fact_checks, pairs)
    mergedata = mergedata.drop_duplicates(subset="translated_claim", keep="first")
    mergedata = mergedata.drop_duplicates(subset="original_claim", keep="first")
    train_df, val_df, test_df = split_data(posts, mergedata)

    return train_df, test_df, val_df, mergedata


import re
import ast
import re
from collections import Counter
import  pandas as pd

class TextPreprocessor:
    @staticmethod
    def remove_urls(text):
        if not isinstance(text, str):
            return text
        return re.sub(r'http\S+', '', text)

    @staticmethod
    def remove_emojis(text):
        if not isinstance(text, str):
            return text
        emoji_pattern = re.compile(
            "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            "]+",
            flags=re.UNICODE,
        )
        return emoji_pattern.sub(r'', text)

    @staticmethod
    def replace_whitespaces(text):
        if not isinstance(text, str):
            return text
        return re.sub(r'\s+', ' ', text).strip()

    def preprocess(self, text):
        text = self.remove_urls(text)
        text = self.remove_emojis(text)
        text = self.replace_whitespaces(text)
        return text

# Move these functions outside of the class

def replace_text_with_ocr(df, text_column='text', ocr_column='ocr'):
    df[text_column] = df.apply(
        lambda row: row[ocr_column] if pd.isna(row[text_column]) or row[text_column].strip() == '' else row[text_column],
        axis=1
    )
    return df


def safe_literal_eval(val):
    try:
        if isinstance(val, str):
            parsed_val = ast.literal_eval(val)
            if isinstance(parsed_val, list):
                for item in parsed_val:
                    if isinstance(item, tuple) and len(item) == 3:
                        return item  # Return the first valid tuple
            elif isinstance(parsed_val, tuple) and len(parsed_val) == 3:
                return parsed_val
        return val
    except (ValueError, SyntaxError) as e:
        print(f"Literal eval failed for {val}: {e}")
        return (None, None, None)


def split_text_column(row, row_name):
    try:
        parsed = safe_literal_eval(row[row_name])
        if isinstance(parsed, tuple) and len(parsed) == 3:
            first_text = parsed[0].strip() if isinstance(parsed[0], str) else None
            second_text = parsed[1].strip() if isinstance(parsed[1], str) else None

            # Extract the language with the highest confidence
            if isinstance(parsed[2], list) and all(isinstance(item, tuple) for item in parsed[2]):
                lang_conf = max(parsed[2], key=lambda x: x[1] if len(x) == 2 else 0)
            else:
                lang_conf = (None, None)

            lang = lang_conf[0]
            confidence = lang_conf[1]
            return pd.Series([first_text, second_text, lang, confidence])
    except Exception as e:
        print(f"Error processing row: {row[row_name]} -> {e}")
    return pd.Series([None, None, None, None])





def tokenize_ranking_data(examples, tokenizer, max_seq_length):
    # Format the texts
    formatted_texts = [
        f"Post: {post}\nClaim: {claim}" for post, claim in zip(examples["post"], examples["claim"])
    ]

    # Tokenize the batch
    tokenized = tokenizer(
        formatted_texts,
        max_length=max_seq_length,
        truncation=True,
        padding="max_length"
    )

    # Add labels
    tokenized["labels"] = examples["label"]
    return tokenized




def format_ranking_data_from_pairs(pairs):
    formatted_data = []
    for _, row in pairs.iterrows():
        formatted_data.append({
            "post": str(row["text_original"]),
            "claim": str(row["original_claim"]),
            "label": int(row["label"])
        })
    return formatted_data



#train_data = format_ranking_data_from_pairs(train_pairs)

#val_data = format_ranking_data_from_pairs(val_pairs)
#test_data = format_ranking_data_from_pairs(test_pairs)

#train_dataset = Dataset.from_list(train_data)
#val_dataset = Dataset.from_list(val_data)
#test_dataset = Dataset.from_list(test_data)



In [15]:
from datasets import Dataset
import pandas as pd

# Preprocessing steps
print(len(posts))
train_df, test_df, val_df, mergedata = pre_process(posts, fact_checks, pairs)
#train_df = train_df.sample(frac=0.5)
#test_df = test_df.sample(frac=0.5)
#val_df = val_df.sample(frac=0.5)
print(len(train_df))

# Generate positive and negative pairs
train_pairs = create_pos_neg_pairs(train_df, mergedata)
val_pairs = create_pos_neg_pairs(val_df, mergedata)
#test_pairs = create_pos_neg_pairs(test_df, mergedata)

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_pairs)
val_dataset = Dataset.from_pandas(val_pairs)
test_dataset = Dataset.from_pandas(test_df)

# Shuffle datasets
train_dataset2 = train_dataset.shuffle(seed=42)
val_dataset2 = val_dataset.shuffle(seed=42)
#test_dataset2 = test_dataset.shuffle(seed=42)

24431
11386


In [22]:
FastLanguageModel.for_inference(model)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
FastLanguageModel.for_inference(model)

# Define the Alpaca-style prompt
alpaca_prompt = """ ### Instruction:
Determine if the following claim is supported by the given post.

### Post:
{}

### Claim:
{}

### Response:
{}."""


def rank_claims(post, claims, model, tokenizer, max_length=512, batch_size=4):

    scores = []
    for i in range(0, len(claims), batch_size):
        batch_claims = claims[i : i + batch_size]
        inputs = [
            alpaca_prompt.format(post, claim, "") for claim in batch_claims
        ]
        
        # Tokenize the batch
        tokenized_inputs = tokenizer(
            inputs,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_length
        ).to("cuda")
        
        # Get logits for the batch
        with torch.no_grad():
            outputs = model(**tokenized_inputs)
            logits = outputs.logits  # Shape: [batch_size, seq_len, vocab_size]

        # Use the last token's logits as relevance scores
        batch_scores = logits[:, -1, tokenizer.convert_tokens_to_ids("Supported")].to(dtype=torch.float32).cpu().numpy()

        scores.extend(batch_scores)
    
    
    ranked_indices = torch.tensor(scores).argsort(descending=True).numpy()  # Descending order
    ranked_claims = [(claims[idx], scores[idx]) for idx in ranked_indices]

    # Return top 10 claims
    return ranked_claims[:10]


posts = test_dataset['text_original'][:500]
claims = test_dataset['original_claim']
claims_orig = test_dataset['original_claim'][:500]


torch.cuda.empty_cache()
success = 0
for post, original_claim in zip(posts, claims_orig):  
    top_10_claims = rank_claims(post, claims, model, tokenizer, batch_size=8)
    print(f"original claim {original_claim}")
    print(f"List of  claim {top_10_claims}")
    for i, (claimfound, score) in enumerate(top_10_claims, start=1):
        if claimfound == original_claim:
            success = success + 1
            print("success")
print(f"success is {success}")



KeyboardInterrupt: 